Исходные данные включают два листа: transaction и customer. В transaction содержится 20000 записей
Обнаруженные проблемы: пропущенные значения в полях last_name, DOB, job_title, job_industry_category, online_order, brand, product_line, product_class и product_size. Также выявлена неконсистентность в product_id, где один идентификатор соответствует разным товарам. Форматы дат требуют преобразования в формат DATE, а числовые поля, такие как postcode, требуют проверки корректности.

Для нормализации базы данных использован стандартный процесс приведения к 3НФ. В первой нормальной форме данные разделены по отдельным сущностям. Во второй нормальной форме устранены зависимости от части первичного ключа, для чего информация о клиентах и товарах вынесена в отдельные таблицы. В третьей нормальной форме удалены транзитивные зависимости, адрес клиента выделен в отдельную таблицу.

Итоговая структура включает таблицы customer, customer_address, product и transaction. В customer хранятся основные данные о клиентах без адресной информации. В customer_address содержатся адресные данные, привязанные к customer_id. В product записаны данные о товарах с созданием синтетического идентификатора. В transaction фиксируются данные о транзакциях с внешними ключами на customer и product.

```https://dbdiagram.io/d
Table customers {
  customer_id int [pk, increment]
  first_name varchar(50) [not null]
  last_name varchar(50) [not null]
  email varchar(100) [unique, not null]
  phone varchar(20)
}

Table customer_addresses {
  address_id int [pk, increment]
  customer_id int [not null]
  street varchar(100) [not null]
  city varchar(50) [not null]
  post_code varchar(20)
  country varchar(50) [not null]
}

Table products {
  product_id int [pk]
  product_name varchar(100) [not null]
  brand varchar(50)
  product_line varchar(50)
  product_class varchar(50)
  product_size varchar(20)
  list_price numeric(10,2) [not null]
}

Table transactions {
  transaction_id int [pk, increment]
  customer_id int [not null]
  product_id int
  transaction_date date [not null]
  amount numeric(10,2) [not null]
}


Ref: customer_addresses.customer_id > customers.customer_id
Ref: transactions.customer_id > customers.customer_id
Ref: transactions.product_id > products.product_id
```



![Схема](schema.png)

![dbeaver](Screenshot_dbeaver.png)

Для создания таблиц в DBeaver написан SQL-скрипт. В нем определены первичные ключи, типы данных и ограничения. Таблица customer включает customer_id, first_name, last_name, gender, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator и owns_car. Таблица customer_address содержит customer_id, address, postcode, state, country и property_valuation. Таблица product имеет product_id, brand, product_line, product_class и product_size. В таблице transaction записаны transaction_id, product_id, customer_id, transaction_date, online_order, order_status, list_price и standard_cost.

```SqL
CREATE SCHEMA IF NOT EXISTS shod;

CREATE TABLE IF NOT EXISTS shod.customer (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    gender VARCHAR(10),
    DOB DATE,
    job_title VARCHAR(100),
    job_industry_category VARCHAR(50),
    wealth_segment VARCHAR(50),
    deceased_indicator CHAR(1),
    owns_car CHAR(3)
);

CREATE TABLE IF NOT EXISTS shod.customer_address (
    customer_id INT PRIMARY KEY REFERENCES shod.customer(customer_id),
    address VARCHAR(255),
    postcode VARCHAR(10),
    state VARCHAR(50),
    country VARCHAR(50),
    property_valuation INT
);

CREATE TABLE IF NOT EXISTS shod.product (
    product_id SERIAL PRIMARY KEY,
    brand VARCHAR(50),
    product_line VARCHAR(50),
    product_class VARCHAR(50),
    product_size VARCHAR(20)
);

CREATE TABLE IF NOT EXISTS shod.transaction (
    transaction_id SERIAL PRIMARY KEY,
    product_id INT REFERENCES shod.product(product_id),
    customer_id INT REFERENCES shod.customer(customer_id),
    transaction_date DATE,
    online_order BOOLEAN,
    order_status VARCHAR(20),
    list_price DECIMAL(10,2),
    standard_cost DECIMAL(10,2)
);
```

Загрузка данных выполнена с помощью Python-скрипта. Он обрабатывает пропуски, преобразует даты в нужный формат, конвертирует online_order в boolean, создаёт уникальный идентификатор для товаров и фильтрует транзакции с отсутствующими customer_id. Данные загружены в PostgreSQL после обработки.

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import numpy as np

file_path = "../customer_and_transaction.xlsx"
xls = pd.ExcelFile(file_path)

df_transaction = pd.read_excel(xls, sheet_name="transaction")
df_customer = pd.read_excel(xls, sheet_name="customer")

df_customer.loc[:, 'last_name'] = df_customer['last_name'].fillna('Unknown')
df_customer.loc[:, 'first_name'] = df_customer['first_name'].fillna('Unknown')

df_customer.fillna({'job_title': 'N/A', 'job_industry_category': 'N/A'}, inplace=True)
df_customer['DOB'] = pd.to_datetime(df_customer['DOB'], errors='coerce')
df_customer['DOB'] = df_customer['DOB'].dt.date

df_transaction['transaction_date'] = pd.to_datetime(df_transaction['transaction_date'], errors='coerce')
df_transaction['transaction_date'] = df_transaction['transaction_date'].dt.date

df_transaction['online_order'] = df_transaction['online_order'].map({'True': True, 'False': False})

df_transaction = df_transaction[df_transaction['customer_id'].isin(df_customer['customer_id'])]

df_transaction['product_unique_id'] = (
    df_transaction.groupby(['brand', 'product_line', 'product_class', 'product_size'])
    .ngroup() + 1
)


product_df = df_transaction[df_transaction['product_unique_id'].notnull()][
    ['product_unique_id', 'brand', 'product_line', 'product_class', 'product_size']
].drop_duplicates()

product_df = product_df.rename(columns={'product_unique_id': 'product_id'})

df_customer_address = df_customer[['customer_id', 'address', 'postcode', 'state', 'country', 'property_valuation']].copy()
df_customer = df_customer.drop(columns=['address', 'postcode', 'state', 'country', 'property_valuation'])

def insert_data(table, dataframe, conn):
    with conn.cursor() as cursor:
        cols = ", ".join(dataframe.columns)
        sql = f"INSERT INTO shod.{table} ({cols}) VALUES %s ON CONFLICT DO NOTHING;"
        print(f"Загрузка данных в {table}, {len(dataframe)} записей...")
        execute_values(cursor, sql, dataframe.replace({np.nan: None}).values.tolist())
    conn.commit()

try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
    )

    insert_data("customer", df_customer, conn)
    insert_data("customer_address", df_customer_address, conn)
    insert_data("product", product_df, conn)
    
    transaction_df = df_transaction[['transaction_id', 'product_unique_id', 'customer_id', 
                                       'transaction_date', 'online_order', 'order_status', 
                                       'list_price', 'standard_cost']].copy()
    transaction_df = transaction_df.rename(columns={'product_unique_id': 'product_id'})
    insert_data("transaction", transaction_df, conn)

    print("Данные успешно загружены в PostgreSQL!")

except Exception as e:
    print(f"Ошибка загрузки данных: {e}")

finally:
    if conn:
        conn.close()
